In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import LSTM, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
data = pd.read_csv('SuperData.csv')

In [4]:
train_sessions, test_sessions = train_test_split(data['label'].unique(), test_size=0.2, random_state=42)
train_data = data[data['label'].isin(train_sessions)]
test_data = data[data['label'].isin(test_sessions)]
# Normalización
scaler = StandardScaler()

# Copia explícita para evitar la advertencia
train_data_copy = train_data.copy()
test_data_copy = test_data.copy()

# Aplicar la transformación a las copias
train_data_copy[['horizontal', 'vertical']] = scaler.fit_transform(train_data_copy[['horizontal', 'vertical']])
test_data_copy[['horizontal', 'vertical']] = scaler.transform(test_data_copy[['horizontal', 'vertical']])

In [5]:
X_train = []
y_train = []
for label, group in train_data.groupby('label'):
    seq = group[['horizontal', 'vertical']].values
    X_train.append(seq)

for label, group in train_data.groupby('label'):
    if 'R' in group['movement_type'].values:
        y_train.append(2)
    elif 'L' in group['movement_type'].values:
        y_train.append(1)
    elif 'U' in group['movement_type'].values:
        y_train.append(3)
    elif 'D' in group['movement_type'].values:
        y_train.append(4)
    else:
        y_train.append(0)

X_test = []
y_test = []
for label, group in test_data.groupby('label'):
    seq = group[['horizontal', 'vertical']].values
    X_test.append(seq)

for label, group in test_data.groupby('label'):
    if 'R' in group['movement_type'].values:
        y_test.append(2)
    elif 'L' in group['movement_type'].values:
        y_test.append(1)
    elif 'U' in group['movement_type'].values:
        y_test.append(3)
    elif 'D' in group['movement_type'].values:
        y_test.append(4)
    else:
        y_test.append(0)

# Convert X_train, y_train, X_test, and y_test to numpy arrays
X_train = np.array(X_train, dtype=object)
# Convert labels to one-hot vectors
y_train = to_categorical(y_train, num_classes=5)

X_test = np.array(X_test, dtype=object)
y_test = to_categorical(y_test, num_classes=5)

In [6]:
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras import regularizers

# Define your model
model = Sequential()
model.add(LSTM(100, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model.add(Dropout(0.5))  # Dropout
model.add(GlobalMaxPooling1D())
model.add(Dense(5, activation='softmax'))

# Compile your model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Pad the sequences to a fixed length
X_train_padded = pad_sequences(X_train, dtype='float32', padding='post')
X_test_padded = pad_sequences(X_test, dtype='float32', padding='post')

# Now, you can feed the padded sequences into your model
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train_padded, y_train, epochs=20, batch_size=64, callbacks=[early_stop])

# Evaluate your model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



Epoch 1/20


4/4 [==============================] - 512s 99s/step - loss: 1.6872 - accuracy: 0.2143
Epoch 2/20
4/4 [==============================] - 475s 110s/step - loss: 1.6073 - accuracy: 0.3010
Epoch 3/20
4/4 [==============================] - 451s 101s/step - loss: 1.5922 - accuracy: 0.3469
Epoch 4/20
4/4 [==============================] - 452s 102s/step - loss: 1.5672 - accuracy: 0.3622
Epoch 5/20
4/4 [==============================] - 456s 102s/step - loss: 1.5194 - accuracy: 0.3878
Epoch 6/20
4/4 [==============================] - 588s 109s/step - loss: 1.4541 - accuracy: 0.4490
Epoch 7/20
4/4 [==============================] - 450s 101s/step - loss: 1.3819 - accuracy: 0.4490
Epoch 8/20
4/4 [==============================] - 451s 100s/step - loss: 1.3531 - accuracy: 0.4133
Epoch 9/20
4/4 [==============================] - 441s 98s/step - loss: 1.3179 - accuracy: 0.4847
Epoch 10/20
4/4 [==============================] - 448s 100s/step - loss: 1.2987 - accuracy: 0.4745
Epoch 1

In [7]:
# Save the model
model.save('my_modelV5.h5')

c:\Users\Asus\Documents\aUniversidad\SEMESTRE 23-2\Bio_I\Proyecto\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
# Load the new dataset
probe_data = pd.read_csv('prueba5.csv')

# Preprocess the probe data in the same way as your training data
X_probe = []
for label, group in probe_data.groupby('label'):
    seq = group[['horizontal', 'vertical']].values
    seq = scaler.transform(seq)  # Normalize the data
    X_probe.append(seq)
X_probe = pad_sequences(X_probe, dtype='float32', padding='post')

# Use the model to predict the class labels for the probe data
y_probe_pred = model.predict(X_probe)

# The output, y_probe_pred, is a 2D array with the probabilities for each class.
# To get the predicted class labels, you can use argmax to find the class with the highest probability.
y_probe_pred_labels = np.argmax(y_probe_pred, axis=1)
print(y_probe_pred_labels)

c:\Users\Asus\Documents\aUniversidad\SEMESTRE 23-2\Bio_I\Proyecto\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 200ms/step
[3]
